In [1]:
import sys
import re
import string
import os
import numpy as np
import codecs
import pickle
import spacy

In [3]:
def split_title(text):
    """
    Given text returns title and the rest of the article.

    Split the test by "\n" and assume that the first element is the title
    """
    ## YOUR CODE HERE
    lines = text.strip().split("\n")
    title = lines[0]
    body = lines[1:]

    return title, body

In [4]:
text = "iraq use members each area found, \nofficial sunday place go based"
title, body = split_title(text)
print(title,type(title), body)

iraq use members each area found,  <class 'str'> ['official sunday place go based']


In [31]:
ENGLISH_STOP_WORDS = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"])


def load_glove(filename):
    """
    Read all lines from the indicated file and return a dictionary
    mapping word:vector where vectors are of numpy `array` type.
    GloVe file lines are of the form:

    the 0.418 0.24968 -0.41242 0.1217 ...

    So split each line on spaces into a list; the first element is the word
    and the remaining elements represent factor components. The length of the vector
    should not matter; read vectors of any length.

    ignore stopwords
    """
    # filename = "test_glove.txt"
    ## YOUR CODE HERE
    glove_dict = {}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().split(' ')
            title = line[0]
            if title in ENGLISH_STOP_WORDS:
                continue
            vector = np.array(line[1:], dtype='float32')
            glove_dict[title]=vector
    return glove_dict

In [6]:
glove_dict = load_glove("test_glove.txt")
print(len(glove_dict))
print(glove_dict)
print(len(glove_dict["meeting"]))

166
{',': array([-2.5539e-01, -2.5723e-01,  1.3169e-01, -4.2688e-02,  2.1817e-01,
       -2.2702e-02, -1.7854e-01,  1.0756e-01,  5.8936e-02, -1.3854e+00,
        5.8509e-01,  3.6501e-02, -1.9846e-01,  1.9613e-01,  4.0929e-01,
        1.5702e-01, -1.5305e-01,  5.0447e-02,  3.0045e-01, -1.1295e-01,
       -1.7043e-02,  1.8593e-01,  1.9982e-01,  2.0053e-01, -6.3141e-01,
       -1.2622e-01,  2.9510e-01, -2.6282e-01, -1.5831e-01,  1.2383e-03,
        1.1784e-02,  5.8758e-01, -1.5914e-01,  2.7731e-01, -8.2343e-01,
       -2.1134e-01,  1.3414e-02,  1.9637e-01, -4.1470e-01,  1.0276e-03,
        1.3422e-01, -1.4205e-01,  5.1545e-02,  3.4993e-01, -2.9868e-01,
       -3.2090e-01,  1.9566e-01,  4.7886e-01,  1.0744e-01,  1.0004e-02,
        1.8503e-01,  8.0694e-02,  2.0739e-01, -9.7365e-02, -3.9448e-02,
        2.0151e-02, -1.7378e-01,  2.5679e-01,  2.4198e-01, -3.5100e-01,
        1.8759e-01,  6.3857e-03,  1.8395e-01, -1.3929e-01,  8.1855e-03,
       -6.3109e-01,  2.9832e-01,  3.1731e-01,  1.3022e

In [9]:
def words(text):
    """
    Given a string, return a list of words normalized as follows.
    
        1. Lowercase all words
        2. Use re.sub function and string.punctuation + '0-9\\r\\t\\n]'
            to replace all those char with a space character.
        3. Split on space to get word list.
        4. Ignore words < 3 char long.
        5. Remove English stop words
    Don't use Spacy.
    """
    text = text.lower()
    text = re.sub("[" + string.punctuation + '0-9\\r\\t\\n]', ' ', text)
    ## YOUR CODE HERE 
    text = text.split(" ")
    text = [word for word in text if len(word)>2]
    words = [word for word in text if word.lower() not in ENGLISH_STOP_WORDS]
    return words

In [10]:
def doc2vec(text, gloves):
    """
    Return the word vector centroid for the text. Sum the word vectors
    for each word and then divide by the number of words. Ignore words
    not in gloves.
    """
    ## YOUR CODE HERE
    # text = "iraq use members each area found, official sunday place go based"
    # gloves = {',': ['-0.25539', '-0.25723', '0.13169'], 'sunny':['-0.25539', '-0.25723', '0.13169']}
    # word vector centroid = sum of word_vectors / len(word)
    word_ls = words(text)
    sum = gloves[word_ls[0]]
    for word in word_ls[1:]:
        sum += gloves[word]
    centroid = sum/len(word_ls)
    return centroid


In [12]:
text = "iraq use members each area found, official sunday place go based"
gloves = load_glove("test_glove.txt")
len(doc2vec(text, gloves))

300

In [35]:
x = np.array([1.5,0,0])
y =0
x+y

array([1.5, 0. , 0. ])

In [3]:
ls = [(2, ['384.txt', 'y', 'y']),(3, ['384.txt', 'y', 'y']),(1, ['3.txt', 'title', 't', 't'])]
ls.sort(key=lambda x: x[0])
print(ls)

[(1, ['3.txt', 'title', 't', 't']), (2, ['384.txt', 'y', 'y']), (3, ['384.txt', 'y', 'y'])]


In [7]:
ls[0][1][0]

'3.txt'

In [11]:
t = "tech/1.txt"
t.split('/')[-2]

'tech'

In [23]:
def split_title(text):
    """
    Given text returns title and the rest of the article.

    Split the test by "\n" and assume that the first element is the title
    """
    ## YOUR CODE HERE
    lines = text.strip().split("\n")
    title = lines[0]
    body = lines[1:]
    body =' '.join(body)
    return title, body

In [24]:
import codecs
def get_text(filename):
    """
    Load and return the text of a text file, assuming latin-1 encoding as that
    is what the BBC corpus uses.  Use codecs.open() function not open().
    """
    f = codecs.open(filename, encoding='latin-1', mode='r')
    s = f.read()
    f.close()
    return s
filename = '/Users/tingpan/data/bbc/business/021.txt'
file_text = get_text(filename)

In [25]:
title, body = split_title(file_text)
body

" Leisure group Rank could unveil plans to demerge its film services unit and sell its media business, reports claim.  Rank, formerly famous for the Carry On series, will expose the shake-up at the announcement of its results on Friday, the Sunday Telegraph reported. Advisors Goldman Sachs are understood to have valued its demerged Deluxe Film unit at Â£300m, the report added. Speculation of a possible shake-up has mounted since Rank announced a study into a possible demerger in September. Since Mike Smith's appointment as chief executive in 1999, the group has focused on fewer businesses and embarked on a major cost-cutting programme which has seen it dispose of a number of businesses, including the Odeon cinema chain and the Pinewood studios. The move left the group with three core divisions: gaming, Hard Rock and Deluxe Films, which provides technical services to Hollywood studios.  Rank now aims to concentrate on its gaming, bars and hotels business, including extending its Hard Ro

In [32]:
def words(text):
    """
    Given a string, return a list of words normalized as follows.
    
        1. Lowercase all words
        2. Use re.sub function and string.punctuation + '0-9\\r\\t\\n]'
            to replace all those char with a space character.
        3. Split on space to get word list.
        4. Ignore words < 3 char long.
        5. Remove English stop words
    Don't use Spacy.
    """
    text = text.lower()
    text = re.sub("[" + string.punctuation + '0-9\\r\\t\\n]', ' ', text)
    ## YOUR CODE HERE 
    text = text.split(" ")
    text = [word for word in text if len(word)>2]
    words = [word for word in text if word.lower() not in ENGLISH_STOP_WORDS]
    return words
word_ls = words(body)

In [34]:
word_ls[0]

'leisure'

In [36]:
from flask import Flask, render_template
import sys
import pickle

In [59]:
def articles():
    """Show a list of article titles"""
    # ## YOUR CODE HERE
    with open("articles.pkl", "rb") as file:
        articles = pickle.load(file)
        return [article[1] for article in articles[2:]]



In [7]:
f = open('articles.pkl', 'rb')
articles = pickle.load(f)
f.close()
articles


[['entertainment',
  '289.txt',
  'Musicians to tackle US red tape',
  ' Musicians\' groups are to tackle US visa regulations which are blamed for hindering British acts\' chances of succeeding across the Atlantic.  A singer hoping to perform in the US can expect to pay $1,300 (Â£680) simply for obtaining a visa. Groups including the Musicians\' Union are calling for an end to the "raw deal" faced by British performers. US acts are not faced with comparable expense and bureaucracy when visiting the UK for promotional purposes.  Nigel McCune from the Musicians\' Union said British musicians are "disadvantaged" compared to their US counterparts. A sponsor has to make a petition on their behalf, which is a form amounting to nearly 30 pages, while musicians face tougher regulations than athletes and journalists. "If you make a mistake on your form, you risk a five-year ban and thus the ability to further your career," says Mr McCune.  "The US is the world\'s biggest music market, which mea

In [5]:
f = open('recommended.pkl', 'rb')
recommended = pickle.load(f)
f.close()

len(recommended)
    # if (topic, filename) in recommended:
    #     rec = recommended[(topic, filename)]
    # else:
    #     rec = []


2225

In [ ]:
# Launch with
#
# python app.py

from flask import Flask, render_template
import sys
import pickle

app = Flask(__name__)

@app.route("/")
def articles():
    """Show a list of article titles"""
    ## YOUR CODE HERE
    return render_template('articles.html', articles = articles)


@app.route("/article/<topic>/<filename>")
def article(topic, filename):
    """
    Show an article with relative path filename. Assumes the BBC structure of
    topic/filename.txt so our URLs follow that.
    """
    # YOUR CODE HERE
    for art in articles:
        if art[0] == topic and art[1] == filename:
            selected_article = art
            break
    
    if (topic, filename) in recommended:
        rec = recommended[(topic, filename)]
    else:
        rec = []

    return render_template('article.html', article=selected_article, recommend=rec)



f = open('articles.pkl', 'rb')
articles = pickle.load(f)
f.close()


f = open('recommended.pkl', 'rb')
recommended = pickle.load(f)
f.close()
"""
 ('entertainment',
  '289.txt'): 
  [['entertainment','131.txt','Franz man seeks government help'], 
   ['politics', '250.txt','What the election should really be about?'], 
   ['entertainment',
   '271.txt',
   "German music in a 'zombie' state"],
"""

# you may need more code here or not


# for local debug
if __name__ == '__main__':
    app.run(debug=True)